In [40]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
#from dbscan_inner import dbscan_inner
from sklearn.cluster import DBSCAN
from time import time

In [145]:
min_samples  = 5;
eps     = 0.2;

In [146]:
filename      = "../ProteinData_ttx_1hr_2/AHA_2_MMStack_Pos0.ome_locs_render_driftcor_filter_render_pix0.02X6f20";
#filename = "../MikeData/EGFR-P1-ATTO655_cell_3_MMImages.ome_locs_render"
#filename = "../MikeData/EGFR-P1-ATTO655_cell_2_MMImages.ome_locs_render_al_linked1sigma"

In [147]:
XC = np.loadtxt(filename+".txt");

In [148]:
X = XC;

In [149]:
def GetCluster_Type1(ind):
    neighbors_model = NearestNeighbors(
        radius=eps, algorithm='auto',
        leaf_size=30, metric='euclidean',
        metric_params=None, p=None, n_jobs=None)
    neighbors_model.fit(X)
    # This has worst case O(n^2) memory complexity
    neighborhoods = neighbors_model.radius_neighbors(X,return_distance=False)

    n_neighbors = np.array([len(neighbors)
                            for neighbors in neighborhoods])

    # A list of all core samples found.
    is_core = np.asarray(n_neighbors >= min_samples,dtype=np.uint8)
    
    labels = np.full(X.shape[0], -1, dtype=np.intp)
    label_num = 0
    stack = [];

    for i in [ind]:
        if labels[i] != -1 or not is_core[i]:
            continue

        # Depth-first search starting from i, ending at the non-core points.
        # This is very similar to the classic algorithm for computing connected
        # components, the difference being that we label non-core points as
        # part of a cluster (component), but don't expand their neighborhoods.
        while True:
            if labels[i] == -1:
                labels[i] = label_num
                if is_core[i]:
                    neighb = neighborhoods[i]
                    for i in range(neighb.shape[0]):
                        v = neighb[i]
                        if labels[v] == -1:
                            stack.append(v)

            if len(stack) == 0:
                break
            i = stack[-1]
            stack = stack[:-1];

        label_num += 1
    
    return labels;

In [150]:
def GetCluster_Type2(ind):
    neighbors_model = NearestNeighbors(
        radius=eps, algorithm='auto',
        leaf_size=30, metric='euclidean',
        metric_params=None, p=None, n_jobs=None)
    neighbors_model.fit(X)
    # This has worst case O(n^2) memory complexity

    #neighborhoods = neighbors_model.radius_neighbors(X,return_distance=False)
#    n_neighbors = np.array([len(neighbors) for neighbors in neighborhoods])

    
    labels    = np.full(X.shape[0], -1, dtype=np.intp)
    label_num = 0
    stack     = [];
    i         = ind;

    neighborhood = neighbors_model.radius_neighbors([X[i,:]],return_distance=False)[0]
    
    if ((len(neighborhood) >= min_samples)):
        # Depth-first search starting from i, ending at the non-core points.
        # This is very similar to the classic algorithm for computing connected
        # components, the difference being that we label non-core points as
        # part of a cluster (component), but don't expand their neighborhoods.
        while True:
            if labels[i] == -1:
                labels[i] = label_num
                if ((len(neighborhood) >= min_samples)):
                    neighb = neighborhood
                    for i in range(neighb.shape[0]):
                        v = neighb[i]
                        if labels[v] == -1:
                            stack.append(v)

            if len(stack) == 0:
                break

            #Update
            i = stack[-1]        
            stack = stack[:-1];
            neighborhood = neighbors_model.radius_neighbors([X[i,:]],return_distance=False)[0]

    
    return labels;

In [151]:
# Test

In [152]:
ind = 2;
t1 = time();
labels = GetCluster_Type1(ind)
print('comp time = '+str(time()-t1))

comp time = 5.170071840286255


In [153]:
ind = 2;
t1 = time();
labels = GetCluster_Type2(ind)
print('comp time = '+str(time()-t1))

KeyboardInterrupt: 

In [154]:
#Test
t1 = time()
DB = DBSCAN(eps=eps,min_samples=min_samples).fit(X);
labels_DBSCAN = DB.labels_;
print('comp time = '+str(time()-t1))


comp time = 4.313690900802612


In [155]:

#Is it recognized as noise /signal?
if( (labels[ind]>-1) == (labels_DBSCAN[ind] > -1)):
    print("Correctly identified");
else:
    print("Not correctly identified as noise / signal");    
    
#Is size of the group correct?
if( np.sum(labels>-1) == np.sum(labels_DBSCAN == labels_DBSCAN[ind])):
    print("Size of cluster correctly identified as "+str(np.sum(labels>-1)));
else:
    print("Size of cluster not correctly identified");    
    

Correctly identified
Size of cluster correctly identified as 575
